In [ ]:
import pandas as pd
metadata = pd.read_csv(r'C:\Users\taniy\Downloads\MovieRecommender\movies_metadata.csv',low_memory = False)
#print(list(metadata.columns))
#['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count']
#print(metadata.size) 1091184
#print(metadata.shape) (45466, 24)
metadata.iloc[0]

   SIMPLE RECOMMENDER

In [ ]:
# Calculating vote average C (average rating) = 5.618207215133889
C = metadata['vote_average'].mean() 
# Minimum number of votes required to be in the chart m = 160
m = metadata['vote_count'].quantile(0.9)
# Filter out movies with number of votes >= 160 to a new table called q_movies
q_movies = metadata.copy().loc[metadata['vote_count']>=m]

In [ ]:
def weighted_rating(X,m=m,C=C):
  v = X['vote_count']
  R = X['vote_average']
  return ((v/(v+m))*R + (m/(v+m))*C)
q_movies['score'] = q_movies.apply(weighted_rating,axis = 1)
q_movies = q_movies.sort_values('score',ascending = False)
q_movies[['title','vote_count','vote_average','score']].head(20)

   CONTENT BASED RECOMMENDER

In [ ]:
print(metadata.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a', 'is'.
tfidf = TfidfVectorizer(stop_words = 'english')
# Replace NaN with an empty string
metadata['overview'] = metadata['overview'].fillna('')
# Row : movie overview and Column : words in the 'overview' vocabulary 
tfidf_matrix = tfidf.fit_transform(metadata['overview'])
# print(tfidf_matrix.shape) =  (45466, 75827)
# print(tfidf_matrix)

# x = tfidf.get_feature_names()   Len(x) = 75827    x is an array that has all the words in the 'overview' vocabulary
tfidf.get_feature_names()[5000:5010]

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)

In [ ]:
cosine_sim.shape

In [ ]:
metadata.shape

In [ ]:
indices = pd.Series(metadata.index, index = metadata['title']).drop_duplicates()

In [ ]:
def get_recommendations(title, cosine_sim):
  idx = indices[title]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores,key = lambda x:x[1],reverse = True)
  sim_scores = sim_scores[1:11]
  movie_indices = [item[0] for item in sim_scores]
  return metadata['title'].iloc[movie_indices]
# get_recommendations('The Dark Knight Rises',cosine_sim)

CREDITS, GENRES, AND KEYWORDS BASED RECOMMENDER

In [ ]:
# Based on the following metadata: the 3 top actors, the director, related genres, and the movie plot keywords.
credits = pd.read_csv(r'C:\Users\taniy\Downloads\MovieRecommender\credits.csv',low_memory = False)
keywords = pd.read_csv(r'C:\Users\taniy\Downloads\MovieRecommender\keywords.csv',low_memory = False)
metadata = metadata[:25000]
credits = credits[:25000]
keywords = keywords[:25000]
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata = metadata.drop([19730])
metadata['id'] = metadata['id'].astype('int')
# metadata.shape = (24999, 35)

In [ ]:
# added 'cast','crew' from credits.csv and 'keywords' from keywords.csv. All 3 csv files contained 'id'.
metadata = metadata.merge(credits,on='id')
metadata = metadata.merge(keywords,on='id')
# metadata.shape = (28383, 35)

In [ ]:
# From the new features; cast, crew, and keywords, you need to extract the three most important actors, the director and the keywords associated with that movie.
from ast import literal_eval
features = ['cast','crew','keywords','genres']
for feature in features:
  metadata[feature] = metadata[feature].apply(literal_eval)

In [ ]:
# From the new features; cast, crew, and keywords, you need to extract the three most important actors, the director and the keywords associated with that movie.

import numpy as np

def get_director(x):
  for i in x:
    if i['job'] == 'Director':
      return i['name']
  return np.nan

# Return the top 3 elements or the entire list, whichever is more. List refers to the cast, keywords, and genres.
def get_list(x):
  if isinstance(x,list):
    names = [i['name'] for i in x]
    if len(names) > 3: #checking if more than 3 elements exists
      names = names[:3]
    return names
  return []

metadata['director'] = metadata['crew'].apply(get_director)
features = ['cast','keywords','genres']
for feature in features:
  metadata[feature] = metadata[feature].apply(get_list)
# metadata[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

In [ ]:
# Convert cast, keywords, genres and director to lower case and remove spaces
def clean_data(x):
  if isinstance(x,list):
    return [str.lower(i.replace(" ","")) for i in x]
  else:
    if isinstance(x,str):
      return str.lower(x.replace(" ",""))
    else:
      return ''

features = ['cast','keywords','genres','director']
for feature in features:
  metadata[feature] = metadata[feature].apply(clean_data)

In [ ]:
def create_soup(x):
  return ' '.join(x['keywords'])+' '+' '.join(x['cast'])+' '+x['director']+' '+' '.join(x['genres'])
metadata['soup'] = metadata.apply(create_soup,axis = 1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])
# count_matrix.shape = (25113, 44751)
# there are 44751 vocabularies in the metadata['soup'] that you fed to it.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim2 = cosine_similarity(count_matrix,count_matrix)

In [ ]:
metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index = metadata['title'])
#get_recommendations('The Dark Knight Rises', cosine_sim2)